import pandas as pd

# Wczytaj plik CSV z dysku Google Drive
books = pd.read_csv("D:/Downloads/archive(4)/Books.csv",low_memory=False)
books.head()

In [2]:
print(books[books['Book-Title'].str.startswith('Harry')][['Book-Title', 'ISBN']])
print(books['ISBN']=="059035342X")

                                               Book-Title        ISBN
2143    Harry Potter and the Sorcerer's Stone (Harry P...  059035342X
2809       Harry Potter and the Sorcerer's Stone (Book 1)  0590353403
3459     Harry Potter and the Chamber of Secrets (Book 2)  0439064872
3839    Harry Potter and the Prisoner of Azkaban (Book 3)  0439136350
5431         Harry Potter and the Goblet of Fire (Book 4)  0439139597
...                                                   ...         ...
257263  Harry Potter and the Goblet of Fire (Harry Pot...  043955490X
259611              Harry Potter and the Sorcerer's Stone  0939173344
260025             Harry Goes to Camp (Monsterkids, No 3)  0061062383
265961       Harry and Catherine (Vintage Contemporaries)  0679730761
266651             Harry Houdini: Escape Artist (Level 2)  0689848153

[144 rows x 2 columns]
0         False
1         False
2         False
3         False
4         False
          ...  
271355    False
271356    False
271357  

In [3]:
books=books[['ISBN','Book-Title']]
#print(books)
books = books.head(100000)
#print(books.head(10))


books.isnull().sum()
books.drop_duplicates()

,ISBN,Book-Title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi
...,...,...
99995,3442727782,Das Tagebuch des Oscar Wilde.
99996,0373707495,Memories Of You (Bestselling Author Flash) (Ha...
99997,3442725739,Ã?Â?stlich der Berge.
99998,3453211901,Das teuflische Testament.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer


books['Book-Title'] = books['Book-Title'].fillna('')
tfidf = TfidfVectorizer(stop_words='english')

In [6]:
tfidf_matrix = tfidf.fit_transform(books['Book-Title'])
print(tfidf_matrix)

  (0, 27245)	0.7023330351390006
  (0, 9306)	0.7118485146099831
  (1, 7747)	0.7484185646842515
  (1, 9283)	0.6632266973184696
  (2, 28076)	0.7071067811865476
  (2, 11671)	0.7071067811865476
  (3, 8347)	0.4068677213910315
  (3, 41928)	0.35745275221795025
  (3, 35050)	0.2520311895296719
  (3, 871)	0.345037232799594
  (3, 29263)	0.3886591553323592
  (3, 20775)	0.3766654794821571
  (3, 18096)	0.2116571765125366
  (3, 37770)	0.18795811463736217
  (3, 16147)	0.3886591553323592
  (4, 41225)	0.7406908781269843
  (4, 27094)	0.6718459816501673
  (5, 42874)	0.5761758719958254
  (5, 17751)	0.5088038510459882
  (5, 22452)	0.6396405284928577
  (6, 20454)	0.47654311673032956
  (6, 19569)	0.4978672969520545
  (6, 26109)	0.4430897447876674
  (6, 16304)	0.5174362650611891
  (6, 43307)	0.24691294454954157
  :	:
  (99994, 4741)	0.4012002857062789
  (99994, 13769)	0.34650539934791597
  (99995, 42892)	0.4925746840648862
  (99995, 28868)	0.48011921512525013
  (99995, 12013)	0.35409173353200124
  (99995, 38595

In [7]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
tfidf_matrix = tfidf.fit_transform(books['Book-Title'])
tfidf_matrix_sparse = csr_matrix(tfidf_matrix)

In [26]:
cos=cosine_similarity(tfidf_matrix[1:10000], tfidf_matrix[:])
cos.shape

MemoryError: Unable to allocate 3.01 MiB for an array with shape (394606,) and data type float64

In [9]:
indices = pd.Series(books.index, index=books['Book-Title']).drop_duplicates()
indices

Book-Title
Classical Mythology                                                                                       0
Clara Callan                                                                                              1
Decision in Normandy                                                                                      2
Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It        3
The Mummies of Urumchi                                                                                    4
                                                                                                      ...  
Das Tagebuch des Oscar Wilde.                                                                         99995
Memories Of You (Bestselling Author Flash) (Harlequin Super Romance , Vol 749)                        99996
Ã?Â?stlich der Berge.                                                                                 99997
Das teuflische Te

In [ ]:
def  myFunc(e):
    return e[1]

In [10]:
def get_recomendations(title,n,cos=cos):
    idx = indices[title]
    scores = list(enumerate(cos[idx]))
    scores = sorted(scores,key=myFunc,reverse=True)
    scores = scores[1:n+1]
    book_idx = [x[0] for x in scores]
    rec = books['Book-Title'].iloc[book_idx]
    return rec

NameError: name 'cos' is not defined

In [ ]:
a=get_recomendations('The Hobbit : The Enchanting Prelude to The Lord of the Rings',5)
print(a.tolist())

In [5]:
import gradio as gr

def greet(name):
    return get_recomendations(name,1).tolist()

demo = gr.Interface(fn=greet, inputs="text", outputs="text")
    
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Tutaj zaczyna się coś ciekawego

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer


wektory_cech = CountVectorizer().fit_transform(books['Book-Title'])
#odleglosci_euklidesowe = euclidean_distances(wektory_cech[0], wektory_cech)
dane = []
for i in range(len(books)):
    odleglosci_euklidesowe_i = euclidean_distances("Lord of roi", wektory_cech)
    dane.append((i, odleglosci_euklidesowe_i))

print("Type:", type(dane))  # Checking the type of ajaa
print("Size:", len(dane))

In [ ]:
from Levenshtein import distance
def closest_title(title):
    # Utwórz DataFrame z indeksem
    m = pd.DataFrame(books.index)
    m['lev'] = m.index.map(lambda x: distance(books['Book-Title'].iloc[x], title))

    # Znajdź najbliższy tytuł na podstawie odległości Levenshteina
    closest_title = books.loc[m.sort_values('lev', ascending=True).index[0], 'Book-Title']

    return closest_title

def n_najmniejszych(lista, n):
    # Utwórz listę indeksów dla elementów różnych od zera
    indeksy_niezerowych_elementow = [indeks for indeks, wartosc in enumerate(lista[0]) if abs(wartosc) > 1e-10]
    
    # Posortuj listę indeksów według wartości
    posortowane_indeksy = sorted(indeksy_niezerowych_elementow, key=lambda x: lista[0][x])
    
    # Ogranicz się do pierwszych n najmniejszych indeksów
    n_najmniejsze_indeksy = posortowane_indeksy[:n]
    
    return n_najmniejsze_indeksy


def get_recomendation(title,n):
    check_book = any(books['Book-Title'].str.contains(title, case=False))
    if check_book == True:
        print("1")
        row_number = books.index[books['Book-Title'].str.contains(title, case=False)].tolist()
        row_number =row_number[0]
        array = dane[row_number][1].tolist()
        title_list=n_najmniejszych(array,n)
        tytul_ksiazki =[]
        for indeks in title_list:
            tytul_ksiazki.append(books["Book-Title"].iloc[indeks])

    else:
        print("2")
        title = closest_title(title)
        row_number = books.index[books['Book-Title'].str.contains(title, case=False)].tolist()
        row_number =row_number[0]
        array = dane[row_number][1].tolist()
        title_list=n_najmniejszych(array,n)
        tytul_ksiazki =[]
        for indeks in title_list:
            tytul_ksiazki.append(books["Book-Title"].iloc[indeks])
            
    return(tytul_ksiazki)

get_recomendation("Dark Tales",4)
#get_recomendation("The Hobbit : The Enchanting Prelude to The Lord of the Rings",4)



# Rekomendaja po Popularności danej książki

In [4]:
#wczytanie danych z ocenami
ratings = pd.read_csv("D:/Downloads/archive(4)/Ratings.csv",low_memory=False)
ratings.head()


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
#połącz danych 
book_rating = pd.merge(books,ratings, on ='ISBN')
book_rating.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,0


In [12]:
# Liczba ocen dla każdej książki
num_book_rating = book_rating.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_book_rating.rename(columns = {'Book-Rating':'num_rating'}, inplace=True)
num_book_rating.head()

,Book-Title,num_rating
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1


In [9]:
# Średnia ocena dla każdej książki
avg_book_rating = book_rating.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_book_rating.rename(columns={'Book-Rating':'avg_rating'}, inplace=True)
avg_book_rating.head()

C:\Users\13luk\AppData\Local\Temp\ipykernel_21520\2411533193.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_book_rating = book_rating.groupby('Book-Title').mean()['Book-Rating'].reset_index()


,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25
1,Always Have Popsicles,0.00
2,Apple Magic (The Collector's series),0.00
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00
4,Beyond IBM: Leadership Marketing and Finance ...,0.00


In [13]:
#połączenie danych sredniej oceny i liczby ocen
books_df = avg_book_rating.merge(num_book_rating, on = 'Book-Title')
books_df.head()

,Book-Title,avg_rating,num_rating
0,A Light in the Storm: The Civil War Diary of ...,2.25,4
1,Always Have Popsicles,0.00,1
2,Apple Magic (The Collector's series),0.00,1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.00,1
4,Beyond IBM: Leadership Marketing and Finance ...,0.00,1
